In [1]:
from dotenv import load_dotenv
import requests
import os
import pandas as pd
import math

In [2]:
load_dotenv()
API_KEY = os.getenv("GOOGLE_API_KEY")
API_URL = "https://places.googleapis.com/v1/places:searchNearby"
PHOTO_API_URL = "https://places.googleapis.com/v1"

In [4]:
df = pd.read_csv("google_places.csv")
df

,id,google_name,disp_name,lat,lon,photo_0,photo_1,photo_2,photo_3,photo_4,photo_5,photo_6,photo_7,photo_8,photo_9
0,349323821,places/ChIJ_b6qJvpYwokR7TFSUhdqGUc,Five Guys,40.762546,-73.976704,places/ChIJ_b6qJvpYwokR7TFSUhdqGUc/photos/AUGG...,places/ChIJ_b6qJvpYwokR7TFSUhdqGUc/photos/AUGG...,places/ChIJ_b6qJvpYwokR7TFSUhdqGUc/photos/AUGG...,places/ChIJ_b6qJvpYwokR7TFSUhdqGUc/photos/AUGG...,places/ChIJ_b6qJvpYwokR7TFSUhdqGUc/photos/AUGG...,places/ChIJ_b6qJvpYwokR7TFSUhdqGUc/photos/AUGG...,places/ChIJ_b6qJvpYwokR7TFSUhdqGUc/photos/AUGG...,places/ChIJ_b6qJvpYwokR7TFSUhdqGUc/photos/AUGG...,places/ChIJ_b6qJvpYwokR7TFSUhdqGUc/photos/AUGG...,places/ChIJ_b6qJvpYwokR7TFSUhdqGUc/photos/AUGG...
1,357618253,places/ChIJcdatubBhwokRNd0ySdodDgg,Hooters of Fresh Meadows,40.740789,-73.784351,places/ChIJcdatubBhwokRNd0ySdodDgg/photos/AUGG...,places/ChIJcdatubBhwokRNd0ySdodDgg/photos/AUGG...,places/ChIJcdatubBhwokRNd0ySdodDgg/photos/AUGG...,places/ChIJcdatubBhwokRNd0ySdodDgg/photos/AUGG...,places/ChIJcdatubBhwokRNd0ySdodDgg/photos/AUGG...,places/ChIJcdatubBhwokRNd0ySdodDgg/photos/AUGG...,places/ChIJcdatubBhwokRNd0ySdodDgg/photos/AUGG...,places/ChIJcdatubBhwokRNd0ySdodDgg/photos/AUGG...,places/ChIJcdatubBhwokRNd0ySdodDgg/photos/AUGG...,places/ChIJcdatubBhwokRNd0ySdodDgg/photos/AUGG...
2,357620442,places/ChIJG8WWttZZwokRBkeFkHkoFv4,Sam Sunny,40.741594,-73.978530,places/ChIJG8WWttZZwokRBkeFkHkoFv4/photos/AUGG...,places/ChIJG8WWttZZwokRBkeFkHkoFv4/photos/AUGG...,places/ChIJG8WWttZZwokRBkeFkHkoFv4/photos/AUGG...,places/ChIJG8WWttZZwokRBkeFkHkoFv4/photos/AUGG...,places/ChIJG8WWttZZwokRBkeFkHkoFv4/photos/AUGG...,places/ChIJG8WWttZZwokRBkeFkHkoFv4/photos/AUGG...,places/ChIJG8WWttZZwokRBkeFkHkoFv4/photos/AUGG...,places/ChIJG8WWttZZwokRBkeFkHkoFv4/photos/AUGG...,places/ChIJG8WWttZZwokRBkeFkHkoFv4/photos/AUGG...,places/ChIJG8WWttZZwokRBkeFkHkoFv4/photos/AUGG...
3,357621192,places/ChIJ00nIlp_2wokROYs-HVvAfj8,United Palace,40.846539,-73.937923,places/ChIJ00nIlp_2wokROYs-HVvAfj8/photos/AUGG...,places/ChIJ00nIlp_2wokROYs-HVvAfj8/photos/AUGG...,places/ChIJ00nIlp_2wokROYs-HVvAfj8/photos/AUGG...,places/ChIJ00nIlp_2wokROYs-HVvAfj8/photos/AUGG...,places/ChIJ00nIlp_2wokROYs-HVvAfj8/photos/AUGG...,places/ChIJ00nIlp_2wokROYs-HVvAfj8/photos/AUGG...,places/ChIJ00nIlp_2wokROYs-HVvAfj8/photos/AUGG...,places/ChIJ00nIlp_2wokROYs-HVvAfj8/photos/AUGG...,places/ChIJ00nIlp_2wokROYs-HVvAfj8/photos/AUGG...,places/ChIJ00nIlp_2wokROYs-HVvAfj8/photos/AUGG...
4,368042980,places/ChIJeaNd3IhZwokRU25epwuJ4aY,Storefront for Art and Architecture,40.721478,-73.997071,places/ChIJeaNd3IhZwokRU25epwuJ4aY/photos/AUGG...,places/ChIJeaNd3IhZwokRU25epwuJ4aY/photos/AUGG...,places/ChIJeaNd3IhZwokRU25epwuJ4aY/photos/AUGG...,places/ChIJeaNd3IhZwokRU25epwuJ4aY/photos/AUGG...,places/ChIJeaNd3IhZwokRU25epwuJ4aY/photos/AUGG...,places/ChIJeaNd3IhZwokRU25epwuJ4aY/photos/AUGG...,places/ChIJeaNd3IhZwokRU25epwuJ4aY/photos/AUGG...,places/ChIJeaNd3IhZwokRU25epwuJ4aY/photos/AUGG...,places/ChIJeaNd3IhZwokRU25epwuJ4aY/photos/AUGG...,places/ChIJeaNd3IhZwokRU25epwuJ4aY/photos/AUGG...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296,1427499791,places/ChIJf9GUrp5ZwokRcRxMvzOronY,Heavenly Market and Deli,40.731441,-73.988241,places/ChIJf9GUrp5ZwokRcRxMvzOronY/photos/AUGG...,places/ChIJf9GUrp5ZwokRcRxMvzOronY/photos/AUGG...,places/ChIJf9GUrp5ZwokRcRxMvzOronY/photos/AUGG...,places/ChIJf9GUrp5ZwokRcRxMvzOronY/photos/AUGG...,places/ChIJf9GUrp5ZwokRcRxMvzOronY/photos/AUGG...,places/ChIJf9GUrp5ZwokRcRxMvzOronY/photos/AUGG...,places/ChIJf9GUrp5ZwokRcRxMvzOronY/photos/AUGG...,places/ChIJf9GUrp5ZwokRcRxMvzOronY/photos/AUGG...,places/ChIJf9GUrp5ZwokRcRxMvzOronY/photos/AUGG...,places/ChIJf9GUrp5ZwokRcRxMvzOronY/photos/AUGG...
297,1427499828,places/ChIJjeTT_DxXwokR1sB-qcPZO8k,Fuel 4 Food Mart,40.743989,-74.063014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
298,1427499831,places/ChIJj7cR5gBZwokRXylE1glTthU,Chipotle Mexican Grill,40.751360,-73.980721,places/ChIJj7cR5gBZwokRXylE1glTthU/photos/AUGG...,places/ChIJj7cR5gBZ

In [17]:
for i in range(10):
    df[f'photo_{i}'] = df[f'photo_{i}'].astype(str)

In [5]:
mismatches = pd.read_csv("osm_google_mismatches.csv")

In [9]:
df.drop(df[df['id'].isin(mismatches['id'])].index, inplace=True)

In [30]:
for index, row in df.iterrows():
    # Check for nan rows and continue
    if row['photo_0'] == 'nan':
        continue
    if os.path.exists(f'./{row['photo_0']}.png'):
        print("fdfsad")

da
da
da
da
da
da
da
da
da
da


In [29]:
s = "lala/lasdlfas/asdfff"
print(s[0:s.rfind('/')+1])

lala/lasdlfas/


In [22]:
if os.path.exists('./places/hello.txt'):
    print('exists')

exists


In [36]:
req = {
    "maxHeightPx": 4000,
    "maxWidthPx": 4000,
    "key": API_KEY
}

for index, row in df.iterrows():
    photo = row['photo_0']

    # Check for nan rows and continue
    if photo == 'nan':
        continue

    # If photo already exists skip
    if os.path.exists(f'./{photo}.png'):
        print("Exists")
        continue
    
    # Photo directory
    photo_dir = photo[0:photo.rfind('/')+1]

    try: 
        response = requests.get(PHOTO_API_URL + "/" + photo + "/media", params=req)
        response.raise_for_status()
    except requests.exceptions.HTTPError as errh:
        print("HTTP Error")
        print(errh.args[0])
    except requests.exceptions.RequestException as e:
        print("Request Exception:", e)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the JSON data from the respons
        os.makedirs(photo_dir, exist_ok=True)
        with open(f"{photo}.png", "wb") as file:
            file.write(response.content)
        # Print or process the data as needed
    else:
        # Print the status code if the request failed
        print(f'Failed to retrieve data: {response.status_code}')
        print(response)

Exists
Exists


KeyboardInterrupt: 